# API 및 크롤링

## 환율정보

In [ ]:
import requests as req
from tqdm import tqdm
import time

In [ ]:
# url = 'https://www.koreaexim.go.kr/site/program/financial/exchangeJSON?authkey=8eOK4WcFolW1hpSC3eJFFF6kwtZM3iRX&data=AP01'
# data = req.get(url).json()
# for v in data:
#     print(v)

In [ ]:
# print(data[-1])

In [ ]:
# usd = data[-1]; 
# print(f'{usd['cur_nm']}의 환율은 {usd['deal_bas_r']} 입니다.')    

In [ ]:
# for v in tqdm(data):
#     if v['deal_bas_r'] == '1':
#         continue
#     print(f'{v['cur_nm']}의 환율은 {v['deal_bas_r']} 입니다.')
#     time.sleep(0.2) # 테스트용 지연

## 비트코인정보

In [ ]:

url = "https://api.bithumb.com/v1/ticker"

headers = {"accept": "application/json"}

resp = req.get(url, headers=headers)

print(resp.text)

In [ ]:
import requests as req
url = 'https://api.bithumb.com/public/ticker/ALL_KRW'
data = req.get(url).json()
btc = data['data']['BTC']
btc

In [ ]:
maxpr = float(btc['max_price']) # 최대가
minpr = float(btc['min_price']) # 최대가
openpr = float(btc['opening_price']) #시작가
bandpr = maxpr - minpr #변화폭
prs = {
    '최대가' : maxpr, 
    '최소가' : minpr, 
    '시작가' : openpr, 
    '변화폭' : bandpr
}
print(prs)
#if openpr + bandpr > maxpr:
print(f'BTC는 {'상승' if openpr + bandpr > maxpr else '하락'}장이다.')

## 미세먼지 정보

In [ ]:
#!pip install --upgrade pip

In [ ]:
#!pip install python-dotenv

In [ ]:
import requests as req
from dotenv import load_dotenv
import os

In [ ]:
#url = 'http://apis.data.go.kr/6260000/AirQualityInfoService/getAirQualityInfoClassifiedByStation?resultType=json&serviceKey=GrzE6lSahh65qTdF%2B5ZL3L0oerDRfmClMgea66%2FeT7QNhsIONhnPjLNcaajESy4n24MAVUXwMYKac%2BykzIO6SA%3D%3D&pageNo=1&numOfRows='

In [ ]:
load_dotenv(dotenv_path='.pt')

key_ori = os.getenv('SER_KEY')
key = req.utils.unquote(key_ori)
print(key)

In [ ]:
url = 'http://apis.data.go.kr/6260000/AirQualityInfoService/getAirQualityInfoClassifiedByStation'
prms = {
    'serviceKey' : key,
    'pageNo' : '1',
    'numOfRows' : 800,
    'resultType' : 'json'
}
resp = req.get(url=url, params=prms).json()
print(resp)

In [ ]:
data = resp['response']['body']['items']['item']
# data

In [ ]:
inp = input('측정 지역 입력:') or '연산동'
for v in data:
    if(v['site'] == inp):
        print(f'\n<<{v['controlnumber']}>>\n{v['site']} 대기질 측정\n미세먼지:{v['pm10']}\n초미세먼지:{v['pm25']}\n{'='*25}\n')



# 네이버 검색 API

## 뉴스

In [ ]:
import os
import sys
import json
import urllib.request
from dotenv import load_dotenv

In [ ]:
load_dotenv(dotenv_path='.pt')

client_id = os.getenv('CLIENT_ID')
# id_ = req.utils.unquote(client_id)
client_secret = os.getenv('CLIENT_SECRET')
# sec_ = req.utils.unquote(client_secret)

encText = urllib.parse.quote(input('검색어 입력'))
url = "https://openapi.naver.com/v1/search/news?query=" + encText # JSON 결과
# url = "https://openapi.naver.com/v1/search/blog.xml?query=" + encText # XML 결과
request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id", client_id)
request.add_header("X-Naver-Client-Secret", client_secret)
response = urllib.request.urlopen(request)
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    data = json.loads(response_body.decode('utf-8'))['items']
    for e in data:
        print('='*115)
        print(f'제목: {e['title']}\n링크: {e['link']}\n내용: {e['description']}\n')
else:
    print("Error Code:" + rescode)

# 크롤링 

## 네이버 주식

In [ ]:
import requests as req
from bs4 import BeautifulSoup as bs

In [ ]:
url = 'https://finance.naver.com/sise/sise_market_sum.naver'
webpage = req.get(url)

# 순수 html 방식
# html = webpage.text
# print(html.find('body'))
# print(html[html.find('body'):(html.find('/html>')-1)])

# soup(bs) 사용
soup = bs(webpage.content, 'html.parser')

# contents = soup.find_all(class_='type_2')
# contents = soup.find_all(class_='tltle')
# for c in contents:
#     print(c.text)
# print(contents)

tbl = soup.find('table', class_='type_2')
tbdy = tbl.find('tbody')
rows = tbdy.find_all('tr')
print('#'*20)
for row in rows:
    if (row.find('a')):
        no = row.find(class_='no').text
        ttl = row.find('a', class_='tltle').text
        rowData = row.find_all(class_='number')
        curr = rowData[0].text
        inca = rowData[1].text.strip()
        fnm = inca.rfind('\t')+1
        inc = ('▲' if inca[:2] == '상승' else ('▼' if inca[:2] == '하락' else '＝')) + inca[3 if not fnm else fnm:]
        per = rowData[2].text.strip()
        
        print(f'No: {no}\n종목: {ttl}\n현재가: {curr}\n전일비: {inc}\n등락률: {per}\n액면가: {rowData[3].text}\n시가총액: {rowData[4].text}\n상장주식수: {rowData[5].text}\n외국인비율: {rowData[6].text}\n거래량: {rowData[7].text}\nPER: {rowData[8].text}\nROE: {rowData[9].text}\n' + '#'*20)
        

## 멜론 차트

In [ ]:
import requests as req
from bs4 import BeautifulSoup as bs

In [ ]:
url = 'https://www.melon.com/chart/index.htm'
headers = {
    'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:133.0) Gecko/20100101 Firefox/133.0'
}
web = req.get(url, headers=headers)
# print(web)
soup = bs(web.content, 'html.parser')
# soup
# print(soup)

curdatetime = soup.select_one('.calendar_prid')
curdate = curdatetime.select_one('.year').text
curtime = curdatetime.select_one('.hour').text

print(f'측정 시각: {curdate} {curtime}')
print('\n\n' + '*'*100)

tbdy = soup.select_one('.type02 table tbody')
# print(tbdy)
# rank = tbdy.select('.rank')
# for a in rank:
#     print(a.text)
# print(rank)
for tr in tbdy.select('tr'):
    rank = tr.select_one('.rank').text # [1].text.strip()
    title = tr.select_one('.rank01 a').text.strip()
    artists = tr.select_one('.rank02 .checkEllipsis').text.strip()
    album = tr.select_one('.rank03').text.strip()
    like = tr.select_one('.like > .cnt').text[4:].strip()
    print(f'{rank} : {title} | {artists} <{album}> ♡{'like'}')


## 부산대 학식

In [ ]:
# !pip install html5lib

In [ ]:
import requests as req
from bs4 import BeautifulSoup as bs

In [ ]:
url = 'https://www.pusan.ac.kr/kor/CMS/MenuMgr/menuListOnBuilding.do?'
headers = {
    'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:133.0) Gecko/20100101 Firefox/133.0'
}
web = req.get(url, headers=headers)
soup = bs(web.content, 'html5lib') # html5lib 는 HTML5의 호환성 적용함
title = soup.select_one('.menu-navi .term').text.strip()
menu = soup.select_one('.menu-tbl.type-day')
print(f' <<{title}>> ')
spline = '-'*(25)
day = menu.select('.day')
date = menu.select('.date')
won = menu.select('li h3')
content = menu.select('li h3+p')
for a, b, c, d in zip(date, day, won, content):
    print(spline)
    print(f'{a.text} ({b.text})')
    print(spline)
    print(f'{c.text.strip()[3:]}')
    print(f'{d.text.strip()}')
    print()
# print(menu)

## 성균관대 식당

In [ ]:
import requests as req
from bs4 import BeautifulSoup as bs

In [ ]:
url = 'https://www.skku.edu/skku/campus/support/welfare_11.do?mode=info&srDt=2024-12-12&srCategory=L&conspaceCd=10201034&srResId=4&srShowTime=W'
headers = {
    'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:133.0) Gecko/20100101 Firefox/133.0'
}
web = req.get(url, headers=headers)
soup = bs(web.content, 'html5lib')

In [ ]:
period = soup.select_one('.date_box .oneday_date').text.replace('\t', '').replace('\n', '')
print(f'|<< {period} >>|')
print()

content = soup.select('.cafeteria_view .weekly_list .weeListWrap')

for item in content:
    print('='*10)
    date = item.select_one('.weeListTit').text.replace('\t', '').replace('\n', '')
    date = f'{date[date.find('(')+1:-1]} ({date[:1]})' 
    print(date)
    print('='*10)
    menu = item.select_one('li > pre').text.strip()
    price = item.select('li')[1].text.strip()
    print(menu)
    print(f'{price}￦')

## 워크24 일자리

In [ ]:
import requests as req
from bs4 import BeautifulSoup as bs

url = 'https://www.work24.go.kr/wk/a/b/1200/retriveDtlEmpSrchList.do?&codeDepth2Info=11000&sortField=DATE&sortOrderBy=DESC&benefitSrchAndOr=O&codeDepth1Info=11000&empTpGbcd=1&siteClcd=all&searchMode=Y&careerTypes=N&academicGbnoEdu=noEdu&enterPriseGbn=all&pageIndex=1'
headers = {
    'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:133.0) Gecko/20100101 Firefox/133.0'
}
kword = 'java|python|spring|javascript'
prm = {
    'resultCnt': '30',
    'currentPageNo': '1',
    'keyword': kword,
    'srcKeyword': kword
}
web = req.get(url, headers=headers, params=prm)
print(web)
soup = bs(web.content, 'html5lib')

srchCnt = soup.select_one('.box_table_hd > .section_left').text.strip()
srchCnt = srchCnt[srchCnt.rfind(' ')+1:]
print(f'총 검색결과: {srchCnt}')
print('+'*60)
tbl = soup.select_one('#contentArea tbody')
rows = tbl.select('tr')
for i, row in enumerate(rows, 1):
    cell = row.select_one('td')
    company = cell.select_one('.box_chk-group a').text.strip()
    titleSelector = cell.select_one('.t3_sb')
    title = titleSelector.text.strip()
    title = title[:title.find('\n')]
    detail = titleSelector.get_attribute_list('href')[0]
    print(f'{i}: {title} / {company} \ndetails=\'{detail}\'\n')
print('+'*60)

# 모듈 테스트

In [ ]:
hi() # NameError

In [9]:
# .py 파일 디렉터리 불러오기
import sys
sys.path.append('/Users/penta/Documents/PythonJupyterlab/py')
# 불러온 디렉터리의 .py 파일을 모듈로 사용하기
import testpython as testi
testi.hi()

hi everyone. 안녕하십니까.


In [10]:
# 다른 .py 파일
import UniversityFoodLists as fl
# print(__name__)
fl.printList()

12.09 (월)
제육덮밥*김가루
팽이버섯된장국
연두부*양념장
콩나물무침
무맛김치
그린샐러드*드레싱2종*토핑1종
6000￦
12.10 (화)
닭가슴살떡볶음
미역국
청포묵김가루무침
마늘쫑무침
깍두기
그린샐러드*드레싱2종*토핑1종
6000￦
12.11 (수)
돼지국밥*당면사리
흑미밥
알감자조림
양파쌈장무침
무맛김치
그린샐러드*드레싱2종*토핑1종
6000￦
12.12 (목)
돈까스돈부리
우동국물
단호박샐러드*아몬드
오복지무침
무맛김치
그린샐러드*드레싱2종*토핑1종
6000￦
12.13 (금)
김치짜글이찌개
김가루밥
치킨탕수육
콩자반
깍두기
그린샐러드*드레싱2종*토핑1종
6000￦


In [ ]:
print(__name__)

In [ ]:
#sample.py
def greet():
    print('Hello from module_example.')

if __name__ == '__main__':
    print('This module is running directly.')
    greet()

In [11]:
import sample as smp
smp.greet()

Hello from module_example.


In [ ]:
# !python py/sample.py

In [8]:
import sys
sys.path.append('/Users/penta/Documents/PythonJupyterlab/py')

import Work24Jobs as wk24
kword = 'java|python|css|html|javascript|nodejs|spring|c|django'
wk24.printResult(15, 1, kword)

<Response [200]>
총 검색결과: 235건
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
1: 전기/전자/조립/제어/PLC /HMI 인원을 구합니다. / 탑스씨앤씨(TOPS C&C) 
details='https://www.work.go.kr/empInfo/empInfoSrch/detail/empDetailAuthView.do?searchInfoType=VALIDATION&callPage=&wantedAuthNo=K131312412120014&rtnTarget=list1&pageIndex=1&rtnUrl=?'

2: 지게차 운전원 모집 / 주식회사진성산업 
details='https://www.work.go.kr/empInfo/empInfoSrch/detail/empDetailAuthView.do?searchInfoType=VALIDATION&callPage=&wantedAuthNo=KJQH002412120001&rtnTarget=list2&pageIndex=1&rtnUrl=?'

3: [IT취업률100프로]무료 취준생모집(국내/일본, 수도권무료숙식 제... / ㈜세종교육 
details='https://www.work.go.kr/empInfo/empInfoSrch/detail/retrivePriEmpDtlView.do?searchInfoType=CJK&iorgGbcd=CJK&wantedAuthNo=48206712&rtnTarget=list3&pageIndex=1'

4: 신입 개발자 채용 - 분당 본사 / (주)포네트 
details='https://www.work.go.kr/empInfo/empInfoSrch/detail/retrivePriEmpDtlView.do?searchInfoType=CSI&iorgGbcd=CSI&wantedAuthNo=49580246&rtnTarget=list4&pageIndex=1'

5: 정보보안 솔루션 연구소 개발자 / (주)노바소프트 
details='h